# FLDAS Explorer Dashboard
Modified: Jun 13, 2019

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np, scipy as sp, pandas as pd, geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True


In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param, Tap, Selection1D, PointerXY, RangeXY
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

import panel as pn
pn.extension()

In [ ]:
# SP_ROOT = Path.home()/'Playground/Semantic_Road/'
# SP_UTILS = SP_ROOT/'scripts'

# # Add the utils directorys to the search path
CURR_UTILS = Path('../utils').absolute()

DIRS2ADD = [CURR_UTILS]#, SP_UTILS]
for UTILS_DIR in DIRS2ADD:
    assert UTILS_DIR.exists()
    if str(UTILS_DIR) not in sys.path:
        sys.path.insert(0, str(UTILS_DIR))
        print(f"Added {str(UTILS_DIR)} to sys.path")

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
Path.ls = lambda x: [o.name for o in x.iterdir()]

In [ ]:
from shapely.geometry import Polygon, Point
from geo_helpers import bounds2poly, crop_gdf_to_bounds, get_polys_at_lonlat
from utils import nprint
from river_helpers import load_river_csvs, get_basin_id


## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
H,W = 800,1000
CURVE_H, CURVE_W = 400, W
opts.defaults(
    
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover'], padding=0.1,
              height=CURVE_H, width=CURVE_W),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Overlay(active_tools=['wheel_zoom']),
    
    opts.Points(active_tools=['wheel_zoom'], tools=['hover','tap']),
    opts.Path(active_tools=['wheel_zoom'], tools=['hover']),

    opts.Polygons(active_tools=['wheel_zoom'], tools=['hover','tap']),
    opts.WMTS(height=H, width=W),

)



## Basemap tile
We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).

In [ ]:
# basemap = gvts.EsriImagery
# basemap
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'

# basemap = gv.tile_sources.EsriImagery
# basemap = gv.tile_sources.EsriUSATopo
# basemap = gv.tile_sources.StamenTerrain
topomap = gv.tile_sources.EsriNatGeo
labelmap = gv.tile_sources.StamenLabels 
basemap = topomap #* labelmap

# river = gv.feature.rivers
# boarders = gv.Feature(cf.BORDERS)
# base = basemap * boarders

---
## Load Datasets

 1. River measurements

In [ ]:
data_root = Path.home()/'data/mint'
data_dir = data_root/'river'
data = load_river_csvs(data_dir)
data['geometry'] = gpd.points_from_xy(data.Longitude, data.Latitude)


In [ ]:
# nprint(len(data), data.head(), data.sample(10))
c = 0
for coord, g in data.groupby( ['Latitude', 'Longitude'] ):
    if c >= 5:
        break
    print(coord, len(g))
    c += 1

In [ ]:
gvd = gv.Dataset(data, kdims=['Latitude', 'Longitude', 'Time'], vdims=['River_Width', 'River_Depth'])
dmap = gvd.to(gv.Points, kdims=['Longitude', 'Latitude'], vdims=['River_Width', 'River_Depth'], 
             dynamic=True)

In [ ]:
dmap.opts(color='River_Width', size='River_Width', cmap='viridis')

- Get unique points: With Avg. Width

In [ ]:
points = []
lats, lons = [], []
avg_ws = []
for (lat,lon), g in data.groupby(['Latitude', 'Longitude']):
    points.append(Point(lon, lat))
    lats.append(lat)
    lons.append(lon)
    avg_ws.append(g.River_Width.mean().item())

In [ ]:
df_avg = pd.DataFrame({'geometry': points,
                           'Longitude': lons,
                           'Latitude': lats,
                           'River_Width_Avg': avg_ws})
df_avg.head()
                           

In [ ]:
gv_avg = gv.Points(df_avg, 
                   ['Longitude', 'Latitude'], 
                   'River_Width_Avg').opts(
    color=dim('River_Width_Avg').norm(), 
    size=dim('River_Width_Avg').norm()*40)

In [ ]:
def get_gv_avg(scale):
    gv_avg = gv.Points(df_avg, 
                       ['Longitude', 'Latitude'], 
                       'River_Width_Avg')
    return gv_avg.opts(color=dim('River_Width_Avg').norm(), 
                       size=dim('River_Width_Avg').norm()*scale)

# parameterized class as a strem
class Scale(param.Parameterized):
    scale = param.Number(default=15, bounds=(10,30))
    
# Add flexibility to set the size of points
scale = Scale()
scale_stream = streams.Params(scale)
dmap_gv_avg = hv.DynamicMap(get_gv_avg, 
                            streams=[scale_stream])
# dmap_gv_avg
    

2. Basins data

In [ ]:
basin_dir = data_root/'hybas_world_lev05_v1c'; assert basin_dir.exists()
bounds = (32.95418, 3.42206, 47.78942, 14.95943)#todo

gdf_basins = crop_gdf_to_bounds( gpd.read_file(basin_dir)[['HYBAS_ID', 'geometry']],
                                bounds,
                                remove_empty=True).reset_index().drop('index', axis=1)
gdf_basins['HYBAS_ID']=gdf_basins.HYBAS_ID.astype(str)
print(len(gdf_basins))
# gdf_basins.head()

In [ ]:
gv_basins = gv.Polygons(gdf_basins, vdims=['HYBAS_ID'])

In [ ]:
# %%opts WMTS [height=H, width=W] Polygons(alpha=0.5)
# basemap * gv_basins * dmap * dmap_gv_avg

3. Assign basin_id to points in river measurement data

In [ ]:
## GeoPandas from river pandas dataframe
bids = []
c = 0
for lon, lat in zip(data.Longitude, data.Latitude):
    bid = get_basin_id(gdf_basins, lon,lat)
    bids.append(bid)
#     if c%30==0: print(lon, lat, bid)
    c += 1
    

In [ ]:
data['HYBAS_ID'] = bids

In [ ]:
# Geopadnas river measurement data
gdf_data = gpd.GeoDataFrame(data)
gdf_data.crs = {'init': 'epsg:4326'}
# gdf_data.head()

---
## Add LatLon Tab selector stream
- Fetch the lat/lon of the mouse click position on the basemap

    1. Define callbacks

In [ ]:
def select_data_at_lonlat(data, lon, lat):
    """
    data: dataFrame or GeoDataFrame
    lat: float
    lon: float
    """
    return data[np.isclose(data.Longitude, lon) & np.isclose(data.Latitude, lat)]

def tseries_from_lonlat(data, lon, lat):
    df = select_data_at_latlon(data, lon, lat)
    curve_w = hv.Curve(df, 'Time', 'River_Width', label='width')
    curve_d = hv.Curve(df, 'Time', 'River_Depth', label='depth')
    return curve_w * curve_d 

def tseries_from_index(data, 
                       index_src, 
                       index):
    if len(index) < 1:
        return hv.Curve([])
    elif len(index) > 1:
        print('Warning: multiple points were selected. Only care for the first point')
        index = index[:1]
              
    lon,lat = index_src.iloc[index].Longitude.item(),index_src.iloc[index].Latitude.item()
    print(lon, type(lon),lat, type(lat))
    df = select_data_at_lonlat(data, lon, lat)
    curve_w = hv.Curve(df, 'Time', 'River_Width', label='width')
    curve_d = hv.Curve(df, 'Time', 'River_Depth', label='depth')
    return curve_w * curve_d 

    2. Register streams

In [ ]:
# stream_xy = PointerXY(name='lonlat', 
#                     x=34., y=9., source=gv_basins)

In [ ]:
stream_tab = Selection1D(name='tab', source=gv_avg, index=[0])
dmap_tab = hv.DynamicMap(lambda index: hv.Div(f'point index:  {index}'),
                        streams=[stream_tab])
dmap_tseries = hv.DynamicMap(lambda index: tseries_from_index(data, df_avg, index),
                             streams=[stream_tab])

In [ ]:
stream_btab = Selection1D(name='btap', source=gv_basins, index=[0])
debug_btab = hv.DynamicMap(lambda index: hv.Div(f'basin index:  {index}'),
                        streams=[stream_btab])
# gv_basins + debug_btab

In [ ]:
min_val = np.min(data[['River_Width', 'River_Depth']].min().to_list())
max_val = np.max(data[['River_Width', 'River_Depth']].max().to_list())

In [ ]:
#overlay
#cmap for points:  plt.cm.gist_earth, plasma, inferno, fire, visidis
scale.sacle=40

app = (
    basemap 
    * gv_avg.opts(cmap='plasma') #* dmap_gv_avg 
    * gv_basins.opts(alpha=0.1)
    + dmap_tseries.opts(
     opts.Curve(framewise=True, 
                show_grid=True, 
                ylim=(min_val,max_val)
               )
 )
).cols(1)



In [ ]:
pane = pn.panel(app)

In [ ]:
pane.servable()